In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

In [3]:
lptp = pd.read_csv("/Users/ahnjaehyuk/Dongguk_Univ/Lab/EyeTracking/State_Prediction/data/merged_output_Laptop.csv")

In [4]:
print(lptp.shape)
lptp.head()

(62061, 29)


,name,inst,image,metaid,r_iris_point,r_iris_shape,r_iris_rotate,r_iris_rx,r_iris_ry,r_iris_cx,...,l_eyelid_pcl,r_center_pcl,l_center_pcl,center_pchead,cam,point,distance_c,distance_d,posture,condition
0,NIA22EYE_S1_001_T1_S01_L_rgb_A_D_T_007.png,L,rgb,1,"[[1037, 425], [1057, 442]]",Ellipse,0.0,10.0,8.5,1047.0,...,"[[916, 439], [920, 441], [923, 443], [929, 444...","[1047, 433.5]","[938, 438]","[0.7004, 0.4219, -0.3298]","[161.43, -88.14, -174.93]","[661, 414]",64.0,64.0,D,A
1,NIA22EYE_S1_001_T1_S01_L_rgb_A_D_T_013.png,L,rgb,1,"[[1038, 425], [1058, 442]]",Ellipse,0.0,10.0,8.5,1048.0,...,"[[915, 439], [919, 442], [922, 443], [928, 445...","[1048, 433.5]","[937.5, 438]","[-0.1852, 0.6894, -0.3321]","[161.41, -88.13, -174.9]","[661, 414]",64.0,64.0,D,A
2,NIA22EYE_S1_001_T1_S01_L_rgb_A_D_T_017.png,L,rgb,1,"[[1038, 425], [1058, 443]]",Ellipse,0.0,10.0,9.0,1048.0,...,"[[916, 439], [919, 442], [923, 443], [928, 445...","[1048, 434]","[938, 438.5]","[-0.2045, 0.8818, -0.3341]","[161.15, -88.13, -174.65]","[661, 414]",64.0,64.0,D,A
3,NIA22EYE_S1_001_T1_S01_L_rgb_A_D_T_025.png,L,rgb,1,"[[1037, 426], [1058, 443]]",Ellipse,0.0,10.5,8.5,1047.5,...,"[[916, 440], [920, 443], [923, 444], [929, 446...","[1047.5, 434.5]","[938, 439.5]","[-0.1783, 0.1386, -0.3415]","[161.16, -88.12, -174.66]","[661, 414]",64.0,64.0,D,A
4,NIA22EYE_S1_001_T1_S01_L_rgb_A_D_T_028.png,L,rgb,1,"[[1037, 426], [1057, 443]]",Ellipse,0.0,10.0,8.5,1047.0,...,"[[915, 441], [919, 443], [922, 444], [928, 446...","[1047, 434.5]","[937.5, 439]","[-0.2567, 0.786, -0.333]","[161.26, -88.12, -174.76]","[661, 414]",64.0,64.0,D,A


In [ ]:
def points(data: pd.DataFrame, n_points: int):
    mem = ['x', 'y']
    rows = []
    rows2 = []

    coord_col = data.columns[1]

    for row in data.itertuples(index=False):
        name = row[0]
        coord_string = str(row[1])
        nums = re.findall('[0-9]+', coord_string)

        if len(nums) == n_points:
            rows.append([name] + nums)
        elif len(nums) != n_points:
            rows2.append([name] + nums)
            
        

    # 열 이름 구성
    columns = ['name'] + [
        f'{coord_col}_{i//2 + 1}_{mem[i % 2]}' for i in range(n_points)
    ]

    df_ = pd.DataFrame(rows, columns=columns)
    df_anomaly = pd.DataFrame(rows2)

    return df_, df_anomaly

df_r_iris_point, r_iris_anomaly = points(lptp[["name", "r_iris_point"]], 4)
df_l_iris_point, l_iris_anomaly = points(lptp[["name", "l_iris_point"]], 4)
df_r_eyelid_pcl, r_eyelid_anomaly = points(lptp[["name","r_eyelid_pcl"]], 32)
df_l_eyelid_pcl, l_eyelid_anomaly = points(lptp[["name",'l_eyelid_pcl']], 32)            


In [6]:
lptp[['name', 'r_iris_rx', 'r_iris_ry', 'r_iris_cx', 'r_iris_cy', 'condition']].merge(df_r_iris_point, on= 'name', how= 'left')\
              .merge(df_l_iris_point, on= 'name', how= 'left')\
              .merge(df_r_eyelid_pcl, on= 'name', how= 'left')\
              .merge(df_l_eyelid_pcl, on= 'name', how= 'left')

,name,r_iris_rx,r_iris_ry,r_iris_cx,r_iris_cy,condition,r_iris_point_1_x,r_iris_point_1_y,r_iris_point_2_x,r_iris_point_2_y,...,l_eyelid_pcl_12_x,l_eyelid_pcl_12_y,l_eyelid_pcl_13_x,l_eyelid_pcl_13_y,l_eyelid_pcl_14_x,l_eyelid_pcl_14_y,l_eyelid_pcl_15_x,l_eyelid_pcl_15_y,l_eyelid_pcl_16_x,l_eyelid_pcl_16_y
0,NIA22EYE_S1_001_T1_S01_L_rgb_A_D_T_007.png,10.0,8.5,1047.0,433.5,A,1037,425,1057,442,...,942,434,934,433,926,435,921,436,918,438
1,NIA22EYE_S1_001_T1_S01_L_rgb_A_D_T_013.png,10.0,8.5,1048.0,433.5,A,1038,425,1058,442,...,942,434,933,434,925,435,920,437,917,438
2,NIA22EYE_S1_001_T1_S01_L_rgb_A_D_T_017.png,10.0,9.0,1048.0,434.0,A,1038,425,1058,443,...,942,434,934,434,926,435,921,437,918,438
3,NIA22EYE_S1_001_T1_S01_L_rgb_A_D_T_025.png,10.5,8.5,1047.5,434.5,A,1037,426,1058,443,...,943,435,934,435,926,436,921,438,919,439
4,NIA22EYE_S1_001_T1_S01_L_rgb_A_D_T_028.png,10.0,8.5,1047.0,434.5,A,1037,426,1057,443,...,942,435,933,435,925,436,920,438,918,439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62056,NIA22EYE_S1_148_T1_S10_L_rgb_S_D_T_292.png,7.5,7.5,1034.5,629.5,S,1027,622,1042,637,...,933,632,925,632,915,633,909,634,904,634
62057,NIA22EYE_S1_148_T1_S10_L_rgb_S_D_T_295.png,NaN,NaN,NaN,NaN,S,NaN,NaN,NaN,NaN,...,934,634,926,635,916,635,909,635,904,635
62058,NIA22EYE_S1_148_T1_S10_L_rgb_S_D_T_302.png,8.0,7.5,1035.0,629.5,S,1027,622,1043,637,...,933,632,924,633,914,633,907,634,903,634
62059,NIA22EYE_S1_148_T1_S10_L_rgb_S_D_T_304.png,8.0,8.0,1034.0,629.0,S,1026,621,1042,637,...,933,634,925,635,916,635,910,635,905,635


In [10]:
tablet = pd.read_csv("/Users/ahnjaehyuk/Dongguk_Univ/Lab/EyeTracking/State_Prediction/data/merged_output_Tablet.csv")

print(tablet.shape)
tablet.head(10)

(59492, 29)


,name,inst,image,metaid,r_iris_point,r_iris_shape,r_iris_rotate,r_iris_rx,r_iris_ry,r_iris_cx,...,l_eyelid_pcl,r_center_pcl,l_center_pcl,center_pchead,cam,point,distance_c,distance_d,posture,condition
0,NIA22EYE_S1_001_T1_S10_T_rgb_A_D_L_004.png,T,rgb,1,"[[535, 459], [564, 481]]",Ellipse,0.0,14.5,11.0,549.5,...,"[[376, 504], [381, 507], [385, 508], [392, 508...","[549.5, 470]","[404, 496]","[12.5471, -7.4259, -0.3193]","[-101.68, 57.44, 81.74]","[968, 417]",19.0,19.0,D,A
1,NIA22EYE_S1_001_T1_S10_T_rgb_A_D_L_014.png,T,rgb,1,"[[538, 457], [567, 480]]",Ellipse,0.0,14.5,11.5,552.5,...,"[[380, 503], [385, 506], [389, 507], [396, 508...","[552.5, 468.5]","[406, 495.5]","[7.1443, 13.6156, -0.8365]","[-101.67, 57.45, 81.75]","[968, 417]",19.0,19.0,D,A
2,NIA22EYE_S1_001_T1_S10_T_rgb_A_D_L_015.png,T,rgb,1,"[[539, 457], [567, 480]]",Ellipse,0.0,14.0,11.5,553.0,...,"[[381, 503], [386, 506], [391, 507], [397, 508...","[553, 468.5]","[408, 496]","[4.8549, 12.7317, -0.3802]","[-101.67, 57.45, 81.74]","[968, 417]",19.0,19.0,D,A
3,NIA22EYE_S1_001_T1_S10_T_rgb_A_D_L_026.png,T,rgb,1,"[[547, 457], [576, 479]]",Ellipse,0.0,14.5,11.0,561.5,...,"[[388, 504], [392, 506], [397, 507], [404, 508...","[561.5, 468]","[415.5, 496]","[2.057, 6.8128, -0.6806]","[-101.67, 57.44, 81.75]","[968, 417]",19.0,19.0,D,A
4,NIA22EYE_S1_001_T1_S10_T_rgb_A_D_L_027.png,T,rgb,1,"[[548, 457], [578, 479]]",Ellipse,0.0,15.0,11.0,563.0,...,"[[390, 503], [394, 505], [399, 506], [406, 507...","[563, 468]","[417, 495.5]","[-4.6089, 10.5907, -0.8359]","[-101.67, 57.44, 81.74]","[968, 417]",19.0,19.0,D,A
5,NIA22EYE_S1_001_T1_S10_T_rgb_A_D_L_035.png,T,rgb,1,"[[556, 457], [585, 479]]",Ellipse,0.0,14.5,11.0,570.5,...,"[[396, 503], [401, 505], [406, 506], [413, 506...","[570.5, 468]","[423.5, 494]","[42.082, -9.9608, 0.0369]","[-101.69, 57.44, 81.73]","[968, 417]",19.0,19.0,D,A
6,NIA22EYE_S1_001_T1_S10_T_rgb_A_D_L_039.png,T,rgb,1,"[[556, 457], [585, 481]]",Ellipse,0.0,14.5,12.0,570.5,...,"[[396, 503], [401, 505], [406, 506], [413, 507...","[570.5, 469]","[425, 495.5]","[42.082, -9.9608, 0.0369]","[-101.69, 57.44, 81.73]","[968, 417]",19.0,19.0,D,A
7,NIA22EYE_S1_001_T1_S10_T_rgb_A_D_L_055.png,T,rgb,1,"[[556, 457], [585, 480]]",Ellipse,0.0,14.5,11.5,570.5,...,"[[396, 503], [401, 506], [406, 507], [413, 507...","[570.5, 468.5]","[425, 495.5]","[42.082, -9.9608, 0.0369]","[-101.69, 57.44, 81.73]","[968, 417]",19.0,19.0,D,A
8,NIA22EYE_S1_001_T1_S10_T_rgb_A_D_L_071.png,T,rgb,1,NaN,NaN,NaN,NaN,NaN,NaN,...,"[[393, 508], [398, 509], [404, 508], [412, 507...",NaN,NaN,"[42.082, -9.9608, 0.0369]","[-101.69, 57.44, 81.73]","[968, 417]",19.0,19.0,D,A
9,NIA22EYE_S1_001_T1_S10_T_rgb_A_D_L_077.png,T,rgb,1,"[[554, 458], [582, 481]]",Ellipse,0.0,14.0,11.5,568.0,...,"[[395, 505], [399, 507], [404, 508], [411, 508...","[568, 469.5]","[423, 496]","[42.082, -9.9608, 0.0369]","[-101.69, 57.44, 81.73]","[968, 417]",19.0,19.0,D,A


In [8]:
df_r_iris_point, r_iris_anomaly = points(tablet[["name", "r_iris_point"]], 4)
df_l_iris_point, l_iris_anomaly = points(tablet[["name", "l_iris_point"]], 4)
df_r_eyelid_pcl, r_eyelid_anomaly = points(tablet[["name","r_eyelid_pcl"]], 32)
df_l_eyelid_pcl, l_eyelid_anomaly = points(tablet[["name",'l_eyelid_pcl']], 32)        

In [9]:
tablet[['name', 'r_iris_rx', 'r_iris_ry', 'r_iris_cx', 'r_iris_cy', 'condition']].merge(df_r_iris_point, on= 'name', how= 'left')\
              .merge(df_l_iris_point, on= 'name', how= 'left')\
              .merge(df_r_eyelid_pcl, on= 'name', how= 'left')\
              .merge(df_l_eyelid_pcl, on= 'name', how= 'left')

,name,r_iris_rx,r_iris_ry,r_iris_cx,r_iris_cy,condition,r_iris_point_1_x,r_iris_point_1_y,r_iris_point_2_x,r_iris_point_2_y,...,l_eyelid_pcl_12_x,l_eyelid_pcl_12_y,l_eyelid_pcl_13_x,l_eyelid_pcl_13_y,l_eyelid_pcl_14_x,l_eyelid_pcl_14_y,l_eyelid_pcl_15_x,l_eyelid_pcl_15_y,l_eyelid_pcl_16_x,l_eyelid_pcl_16_y
0,NIA22EYE_S1_001_T1_S10_T_rgb_A_D_L_004.png,14.5,11.0,549.5,470.0,A,535,459,564,481,...,409,488,398,490,387,494,381,498,378,501
1,NIA22EYE_S1_001_T1_S10_T_rgb_A_D_L_014.png,14.5,11.5,552.5,468.5,A,538,457,567,480,...,412,487,401,489,391,493,385,498,382,501
2,NIA22EYE_S1_001_T1_S10_T_rgb_A_D_L_015.png,14.0,11.5,553.0,468.5,A,539,457,567,480,...,413,487,402,489,392,493,386,497,383,500
3,NIA22EYE_S1_001_T1_S10_T_rgb_A_D_L_026.png,14.5,11.0,561.5,468.0,A,547,457,576,479,...,421,487,409,489,399,493,393,498,390,501
4,NIA22EYE_S1_001_T1_S10_T_rgb_A_D_L_027.png,15.0,11.0,563.0,468.0,A,548,457,578,479,...,422,487,411,488,401,492,395,497,392,500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59487,NIA22EYE_S1_148_T1_S08_T_rgb_S_D_T_284.png,14.5,14.0,1083.5,486.0,S,1069,472,1098,500,...,918,483,904,483,890,485,881,489,875,491
59488,NIA22EYE_S1_148_T1_S08_T_rgb_S_D_T_292.png,14.5,14.0,1082.5,486.0,S,1068,472,1097,500,...,917,483,903,482,889,484,880,488,874,490
59489,NIA22EYE_S1_148_T1_S08_T_rgb_S_D_T_300.png,15.0,14.0,1083.0,487.0,S,1068,473,1098,501,...,917,483,903,482,889,485,880,488,875,491
59490,NIA22EYE_S1_148_T1_S08_T_rgb_S_D_T_301.png,14.0,14.5,1083.0,487.5,S,1069,473,1097,502,...,917,482,903,481,889,483,881,487,875,489
